In [1]:
from fastapi import FastAPI, HTTPException, UploadFile, File
from fastapi.responses import JSONResponse
from dotenv import load_dotenv
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import os

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.schema import BaseOutputParser

from concurrent.futures import ThreadPoolExecutor

In [2]:
load_dotenv()


class MedicalBot:
    def __init__(self):
        self.index = None
        self.model_path = "C:\\Users\\Public\\Rag_based_chatbot\\Medical_bot\\Models\\medgemma-4b-it-medical-gguf\\medgemma-4b-it-finnetunned-merged_new_for_cpu_q5_k_m.gguf"
        self.num_threads = os.cpu_count()
        self.conversational_memory = {}
        
        self.executor = ThreadPoolExecutor(max_workers=4)

        self.initialize_services()
    
    def initialize_services(self):
        self.llm = LlamaCpp(
            model_path=self.model_path,
            n_ctx=100000,       # Context size
            n_threads=self.num_threads,
            n_batch=512,      # Increases throughput
            verbose=False,
            temperature=0.7,
            top_p=0.9,
            max_tokens=256,
            stop=["<end_of_turn>"])
        
        self.memory = ConversationBufferWindowMemory(
             k = 5,
             return_messages=True,
             memory_key='chat_history'
        )

        self.prompt_template = PromptTemplate(
            input_variables=["chat_history", "input"],
            template="""You are a helpful medical assistant. Use the conversation history to provide contextual responses.

Chat History:
{chat_history}

<start_of_turn>user
{input}<end_of_turn>
<start_of_turn>assistant
"""
        )

        self.conversation_chain = ConversationChain(
            llm=self.llm,
            memory=self.memory,
            prompt=self.prompt_template,
            verbose=True,
            output_parser=MedicalChatBotOutputparser()
        )

    def chat(self, user_input:str)-> str:
        try:
            response = self.conversation_chain.predict(input=user_input)
            return response
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
    

class MedicalChatBotOutputparser:

    def parse(self, text: str)->str:
            cleaned_text = text.strip()
            return cleaned_text
    
    @property
    def _type(self)-> str:
         return 'medgemma_output_parser'



In [3]:
app = FastAPI(title='Medical_bot')
app.add_middleware(
    CORSMiddleware, 
    allow_origins =[
        'http://loacalhost:3000'
    ],
    allow_credentials=True,
    allow_methods=["GET", "POST", "OPTIONS"],  # Include OPTIONS for preflight
    allow_headers=["Content-Type", "Authorization"],  # List expected headers
)

class QueryHandler(BaseModel):
    query: str

@app.get('/')
async def home_page():
    return {'message':" Application is running"}

@app.post('/chat')
async def chat_page(req: QueryHandler):
    query = req.query
    med = MedicalBot()
    res = await med.chat(query)
    return res

import uvicorn
import nest_asyncio
nest_asyncio.apply()
if __name__ =='__main__':
    uvicorn.run(app, host='127.0.0.1', port=8000)
# it is a classic issue when you're trying to run uvicorn.run() inside an environment that already has an active event loop—like Jupyter Notebook, IPython, or certain IDEs.

INFO:     Started server process [912]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:65134 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:65134 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:65135 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:65135 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:65144 - "POST /chat HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Li